In [ ]:
# Use

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
import os


In [2]:
tqdm.pandas()

In [ ]:
!ls | grep bios


In [4]:

df = pd.read_csv('../data/raw/yale_bios.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 2 columns):
Unnamed: 0    8618 non-null float64
0             8535 non-null object
dtypes: float64(1), object(1)
memory usage: 134.7+ KB


In [5]:
df.head()

,Unnamed: 0,0
0,0.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
1,1.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
2,10.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
3,100.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
4,1000.0,"<div class=""bio-wrap clearfix"" data-module=""st..."


We'll need to break up the strings into blocks and only use the markup that we actually need.

In [ ]:
# the output needs to be handled 1 at a time, and then dumped out carefully.
# df['soup'] = df['0'].progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

In [ ]:
# tSoup = df['0'].head().progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

In [ ]:
# ts = tSoup.iloc[0]

In [6]:
def tableToDict(soup):
    "Given a headerless table, return a dict with all the fields"
    table = {}
    for row in soup.find_all('tr'):
        cells = row.find_all('td')
        table[str(cells[0].text.strip())] = cells[1].text.strip()
        
    return table
    
def soupToDict(soup):
    "Given a player's soup blob, parse their header"
    playerData = tableToDict(soup.select('div.player-info')[0].find('table'))
    playerData['name'] = soup.select('div.player-name span.name')[0].text.strip()
    return playerData
def getSynopsis(soup):
    "Given a player's soup, split their bio information into \
    chunks for each strong header"
    
    # Get all top level children
#     bioPs = soup.select('div.synopsis')[0].find_all('p', recursive=False)
    
#     blocks= []
#     # if their blocks are malformed, skip that block.
#     for i, val in enumerate(bioPs):
#         try:
#             blocks[val.select('strong')[0].text] = val.get_text()
#         except:
#             printf("Text not detected")
        
    
    # Given the variation in blob types, just return
    # single blob for now.
    # Ryan Brenner: b instead of strong tags
    # Jackson Stallings (Junior Year) - UL blobs mixed with P tags
    # Jackson Stallings (Freshman) - Clean example
    
    # Sometimes, no bio
#     http://yalebulldogs.com/sports/c-sail/2016-17/bios/buehler_patrick_nu3o?view=news
    bio = soup.find('div',class_='synopsis')
    return bio.get_text() if bio else None

In [ ]:
# ts.select('div.synopsis')[0].find_all('b')

In [ ]:
# getSynopsis(ts)

In [ ]:
# soupToDict(ts)

In [ ]:
# df['types'] = df['0'].map(type)
# df['types'].describe()

In [ ]:
# df['types']

In [8]:
df['headers'] = df['0'].progress_apply(lambda x: \
                                       soupToDict(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

100%|█| 8618/8618 [02:55<00:00, 49.05it/s]


In [7]:
import json

In [9]:
df['headers'] = df['headers'].progress_apply(lambda x: json.dumps(x, encoding="utf-8"))

100%|█| 8618/8618 [00:00<00:00, 41038.09it/s]


In [10]:
df['synopsis'] = df['0'].progress_apply(lambda x: 0 )

100%|█| 8618/8618 [00:00<00:00, 410382.74it/s]


In [ ]:
# df['synopsis'] = df['0'].progress_apply(lambda x: getSynopsis(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

In [11]:
linkSeries = pd.read_json('bio_urls', typ='series')

In [12]:
linkSeries.head()

0                /sports/m-basebl/mtt/brenner_ryan00.html
1                /sports/m-basebl/mtt/castaldi_joe00.html
10      /sports/m-crewlt/2015-16/bios/straus-goldfarb_...
100     /sports/m-crewlt/2013-14/bios/wulbern_robert_4pfm
1000     /sports/m-footbl/2007-08/bios/coombs_matt00.html
dtype: object

In [13]:
df.index = linkSeries

In [14]:
df.index.rename('url',inplace=True)

In [15]:
df[['synopsis', 'headers']].head()

,synopsis,headers
url,,
/sports/m-basebl/mtt/brenner_ryan00.html,0,"{""Position:"": ""C"", ""Height:"": ""5-11"", ""Year:"":..."
/sports/m-basebl/mtt/castaldi_joe00.html,0,"{""Position:"": ""P"", ""Height:"": ""6-2"", ""Year:"": ..."
/sports/m-crewlt/2015-16/bios/straus-goldfarb_adam_sjz4,0,"{""Hometown:"": ""Princeton, N.J."", ""Major:"": ""Ph..."
/sports/m-crewlt/2013-14/bios/wulbern_robert_4pfm,0,"{""name"": ""Robert Wulbern"", ""Year:"": ""Jr."", ""Ma..."
/sports/m-footbl/2007-08/bios/coombs_matt00.html,0,"{""Position:"": ""DB"", ""Year:"": ""Senior"", ""Height..."


In [16]:
df.head()

,Unnamed: 0,0,headers,synopsis
url,,,,
/sports/m-basebl/mtt/brenner_ryan00.html,0.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{""Position:"": ""C"", ""Height:"": ""5-11"", ""Year:"":...",0
/sports/m-basebl/mtt/castaldi_joe00.html,1.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{""Position:"": ""P"", ""Height:"": ""6-2"", ""Year:"": ...",0
/sports/m-crewlt/2015-16/bios/straus-goldfarb_adam_sjz4,10.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{""Hometown:"": ""Princeton, N.J."", ""Major:"": ""Ph...",0
/sports/m-crewlt/2013-14/bios/wulbern_robert_4pfm,100.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{""name"": ""Robert Wulbern"", ""Year:"": ""Jr."", ""Ma...",0
/sports/m-footbl/2007-08/bios/coombs_matt00.html,1000.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{""Position:"": ""DB"", ""Year:"": ""Senior"", ""Height...",0


In [18]:
df[['synopsis', 'headers']].to_csv('../data/interim/yale_bio_parsed.csv', encoding='utf-8')

In [ ]:
!ls -lS | grep csv.gz